<a href="https://colab.research.google.com/github/davide1536/minesweeper/blob/main/mineSweeper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import random 
import numpy as np
internalRepresentation = []

In [ ]:
def codeProposition(value1, value2):
  pass

In [ ]:
def findSafeCell(board):
  pass

In [ ]:
def createKnowledge(cell, board):
  pass

In [ ]:
def checkTrivialSafeCell(board):
  pass

In [ ]:
def selectNextSafeCell(board):
  nextCell = checkTrivialSafeCell(board)
  #if there aren't any trivial safe cell use sat solver on clause
  if nextCell == 0:
    nextCell = findSafeCell(board)
    
  return nextCell


In [ ]:
def countMines(row,col,board, size):
  counter = 0

  for i in range(3):
    for j in range(3):
      if (row-1+i >= 0 and row-1+i <= size-1) and (col-1+j >= 0 and col-1+j <= size-1):
        if board[row-1+i, col-1+j] == -1:
          counter += 1
        
     
  
  return counter

In [ ]:
#function that, for each cell, counts the number of mines around it
def fillBoard(board, size):
  
  for row in range(size):
    for col in range(size):
      if board[row,col] != -1:
        n_mines = countMines(row, col, board, size)
        board[row,col] = n_mines
    
      
    
  return board


In [ ]:

def buildBoard(size, probability):
  total_mines = 0
  #board[i][j] = 0 --> no bomb
  #board[i][j] = 1 --> bomb
  print("inizializzo board di gioco")
  board = np.zeros((size,size))
  for row in range(size):
    for col in range(size):
      randomNumber = random.uniform(0,1)
      if randomNumber <= probability:
        board[row,col] = -1
        total_mines += 1
      else:
        board[row,col] = 0


  board = fillBoard(board, size)
  print("board di gioco inizializzata con ",total_mines, " ","mine")
  return board
      

In [ ]:
#build a 9x9 board
SIZE = 9
bomb_probability = 1/6
board = buildBoard(SIZE, bomb_probability)
print(board)

inizializzo board di gioco
board di gioco inizializzata con  14   mine
[[ 0.  0.  0.  1. -1.  1.  1. -1. -1.]
 [ 0.  0.  0.  2.  2.  2.  1.  3. -1.]
 [ 0.  0.  0.  2. -1.  3.  1.  1.  1.]
 [ 1.  1.  1.  3. -1. -1.  1.  0.  0.]
 [ 2. -1.  1.  2. -1.  4.  2.  1.  0.]
 [-1.  2.  1.  1.  1.  2. -1.  2.  1.]
 [ 1.  1.  0.  0.  0.  1.  3. -1.  2.]
 [ 0.  1.  1.  1.  0.  0.  2. -1.  2.]
 [ 0.  1. -1.  1.  0.  0.  1.  1.  1.]]


In [4]:
class Cell:
  def __init__(self) -> None:
    self.safe = 0
    self.value = 0
    self.proposition = 0

In [ ]:
def createCell(value, row, col):
  cell = Cell()
  cell.value = value
  cell.safe = 1
  cell.proposition = codeProposition(row,col)

In [ ]:
def startGame(board):
  result = ""
  firstCell = createCell(board[0,0], 0,0)
  if firstCell.value == -1:
    result = "lose"
  else:
    createKnowledge(firstCell, board)

  while not(result == "win") and not(result == "lose"):
    
    cell = selectNextSafeCell(board)
    if cell.value == -1:
      result = "lose"
    else:
      createKnowledge(cell, board)


  return result

In [ ]:
#start game from the up-left corner
result = startGame(board)
print(result)      

lose
